# Imports

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import EfficientNetB0

import os
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

2025-07-26 11:25:48.416951: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-26 11:25:48.426235: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-26 11:25:48.453273: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-26 11:25:48.485574: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753543548.545626  190598 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753543548.56

# Augmentations, Preprocessing and Preparation

In [2]:
data_dir = 'data/RecyclableAndHouseholdWasteClassification/images'

image_size = (256, 256) # also try training using 256x256 images
batch_size = 32 # change this to 16, 32, 64, or 128 and compare the results
seed = 1337 # for repeatability in train/validation split

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0, # normalize pixels
    validation_split=0.2, # 80/20 percent split
    rotation_range = 20, # rotate images 20 degrees CW or CCW
    width_shift_range = 0.1, # shift images up or down 10 %
    height_shift_range = 0.1, # shift images right or left 10%
    shear_range = 0.1, #shears (distorts) images
    zoom_range = 0.15, # zoom in or out
    horizontal_flip = True,
    brightness_range = [0.8, 1.2], # mimics real-world brightness inconsistency 
    fill_mode = 'nearest' # fill in new pixels during augumentations 
)

valid_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split = 0.2
    
)

# load train/valid data

train_data = train_datagen.flow_from_directory(data_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', subset='training', shuffle = True, seed = seed)

valid_data = valid_datagen.flow_from_directory(data_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', subset='validation', shuffle = False, seed = seed)

num_classes = len(train_data.class_indices)

Found 12000 images belonging to 30 classes.
Found 3000 images belonging to 30 classes.


# Build the Model off a EfficientNetB0 model

In [3]:
base_model = EfficientNetB0(
    include_top = False, 
    weights = 'imagenet',
    input_shape=(image_size[0], image_size[1], 3) #keep size always consistent
)

base_model.trainable = False

inputs = layers.Input(shape=(image_size[0], image_size[1], 3))

x = base_model(inputs)
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x) # drop 40% of neurons; avoid overfitting

outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs=inputs, outputs=outputs, name="efficientnetb0_transfer")
model.summary()

2025-07-26 11:25:56.088758: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "efficientnetb0_transfer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 8, 8, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 30)             │        38,430 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,093,121 (15.61 MB)

 Trainable params: 40,990 (160.12 KB)

 Non-trainable params: 4,052,131 (15.46 MB)

# Compile the Model

In [4]:
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the Model

In [5]:
early_stop = EarlyStopping(
    monitor = 'val_loss', # monitor for validaiton loss
    patience = 5, #wait for five consecutive epochs without improvement
    restore_best_weights=True # bring back model with best val_loss    
)

reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss', 
    factor = 0.5, # cut LR in half when called
    patience = 2,
    min_lr=1e-6,
    verbose = 1
)

checkpoint = ModelCheckpoint(
    filepath='best_efficientnetb0.keras',
    monitor='val_accuracy', # monitor for best validation accuracy
    save_best_only=True,
    verbose=1
)

callbacks = [early_stop, reduce_lr, checkpoint]

initial_epochs = 20

history = model.fit(
    train_data,
    epochs=initial_epochs, 
    validation_data = valid_data, 
    callbacks = callbacks,
    verbose = 1
)

unfreeze_from = len(base_model.layers) - 20 # keep most of it frozen, except for 20.

# set trainalble = True to the last 20 of the EfficientNetB0.
for layer_index, layer in enumerate(base_model.layers):
    if layer_index >= unfreeze_from:
        layer.trainable = True  # allow training on these high-level layers
    else:
        layer.trainable = False # keep the lower layers frozen
        
model.compile(optimizer=Adam(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

/home/adi/Congressional-App-Challenge-Backend/venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 551ms/step - accuracy: 0.0369 - loss: 3.7146
Epoch 1: val_accuracy improved from -inf to 0.06533, saving model to best_efficientnetb0.keras
375/375 ━━━━━━━━━━━━━━━━━━━━ 250s 648ms/step - accuracy: 0.0370 - loss: 3.7146 - val_accuracy: 0.0653 - val_loss: 3.3848 - learning_rate: 0.0010
Epoch 2/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 535ms/step - accuracy: 0.0486 - loss: 3.6465
Epoch 2: val_accuracy improved from 0.06533 to 0.07600, saving model to best_efficientnetb0.keras
375/375 ━━━━━━━━━━━━━━━━━━━━ 237s 631ms/step - accuracy: 0.0486 - loss: 3.6464 - val_accuracy: 0.0760 - val_loss: 3.3441 - learning_rate: 0.0010
Epoch 3/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 545ms/step - accuracy: 0.0432 - loss: 3.6069
Epoch 3: val_accuracy did not improve from 0.07600
375/375 ━━━━━━━━━━━━━━━━━━━━ 242s 646ms/step - accuracy: 0.0432 - loss: 3.6068 - val_accuracy: 0.0673 - val_loss: 3.3341 - learning_rate: 0.0010
Epoch 4/20
362/375 ━━━━━━━━━━━━━━━━━━━━ 7s 584ms/step - a

KeyboardInterrupt: 

# Validation for accuracy

In [ ]:
fine_tune_epochs = 10

history_filetune = model.fit(
    train_data,
    epochs=fine_tune_epochs,
    validation_data=valid_data,
    callbacks=callbacks
)

model.load_weights('best_efficientnetb0.keras')

Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 581ms/step - accuracy: 0.0322 - loss: 3.6477
Epoch 1: val_accuracy did not improve from 0.09033
375/375 ━━━━━━━━━━━━━━━━━━━━ 253s 674ms/step - accuracy: 0.0322 - loss: 3.6477 - val_accuracy: 0.0333 - val_loss: 3.4484 - learning_rate: 1.0000e-04
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 528ms/step - accuracy: 0.0347 - loss: 3.6008
Epoch 2: val_accuracy did not improve from 0.09033
375/375 ━━━━━━━━━━━━━━━━━━━━ 228s 608ms/step - accuracy: 0.0347 - loss: 3.6007 - val_accuracy: 0.0407 - val_loss: 3.3706 - learning_rate: 1.0000e-04
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 619ms/step - accuracy: 0.0399 - loss: 3.5709
Epoch 3: val_accuracy did not improve from 0.09033
375/375 ━━━━━━━━━━━━━━━━━━━━ 275s 733ms/step - accuracy: 0.0399 - loss: 3.5709 - val_accuracy: 0.0437 - val_loss: 3.3803 - learning_rate: 1.0000e-04
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 657ms/step - accuracy: 0.0350 - loss: 3.5396

KeyboardInterrupt: 

# Use the Model to Make a Prediction for a user uploaded Image

In [ ]:
def predict_single_image(image_path, model, train_data, image_size):
    
    pil_img = image.load_img(path=image_path, target_size=image_size)
    
    img_array = image.img_to_array(img = pil_img)
    
    img_array = img_array / 255.0 #normalize
    
    img_array = np.expand_dims(a=img_array, axis=0)
    
    prediction_array = model.predict(x=img_array, verbose = 1)
    
    predicted_class_index_array = np.argmax(a=prediction_array, axis=1)
    
    predicted_class_index = int(predicted_class_index_array[0])
    
    class_indices = train_data.class_indices
    index_to_class = {}
    
    for class_name, class_id in class_indices.items():
        index_to_class[class_id] = class_name
        
    predicted_label = index_to_class[predicted_class_index]
    predicted_confidence = float(prediction_array[0][predicted_class_index])
    
    print("Image path is: ", image_path)
    print("Predicted class index: ", predicted_class_index)
    print("Predicted class label: ", predicted_label)
    print("Predicted confidence: ", f"{predicted_confidence:.4f}")
    
    return predicted_label, predicted_confidence

predict_single_image(image_path = "test_imgs/eggshells.jpg", model = model, train_data = train_data, image_size = image_size)